In [1]:
import numpy as np
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.colors import ListedColormap, BoundaryNorm
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from matplotlib.ticker import FixedLocator
import imageio
import os

# 读取数据
nc_path = "/home/Data_Pool/zhaolx/AGCPPD/2022/20220101.nc"
with Dataset(nc_path, 'r') as ds:
    lat = ds.variables['latitude'][:]
    lon = ds.variables['longitude'][:]
    # 取 8 个时次
    clp_all = ds.variables['clp'][0:8, :, :]
Lon2d, Lat2d = np.meshgrid(lon, lat)

# 自定义带时间标注的绘图函数
def plot_classification_frame(data, Lon2d, Lat2d, timestamp, save_path):
    data_masked = np.ma.masked_invalid(data)
    colors = [plt.cm.tab10(i) for i in range(3)]
    cmap = ListedColormap(colors)
    cmap.set_bad(color='white')
    norm = BoundaryNorm(boundaries=[-0.5,0.5,1.5,2.5], ncolors=cmap.N)

    fig = plt.figure(figsize=(12, 6), dpi=200)
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_extent([-180, 180, -70.1, 70.1],
                  crs=ccrs.PlateCarree())
    im = ax.pcolormesh(Lon2d, Lat2d, data_masked, cmap=cmap, norm=norm, shading='auto')
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS, linewidth=0.5)
    ax.add_feature(cfeature.LAND, facecolor='white')
    ax.add_feature(cfeature.OCEAN, facecolor='white')

    gl = ax.gridlines(draw_labels=False, linewidth=0.5,
                      color='gray', alpha=0.5, linestyle='--')
    gl.top_labels = gl.right_labels = False
    gl.xlabel_style = gl.ylabel_style = {'size': 10}
    lon_ticks = list(range(-180, 181, 30))   # [-180, -150, …, 150, 180]
    lat_ticks = [70,50,30,10,-10,-30,-50,-70]
    ax.set_xticks(lon_ticks, crs=ccrs.PlateCarree())
    ax.set_yticks(lat_ticks, crs=ccrs.PlateCarree())

    ax.xaxis.set_major_formatter(LongitudeFormatter())
    ax.yaxis.set_major_formatter(LatitudeFormatter())

    # 时间标注
    ax.text(0.01, 0.97, timestamp,
            transform=ax.transAxes,
            fontsize=14,
            color='black',
            bbox=dict(facecolor='white', alpha=0.7, edgecolor='none', pad=3))

    cbar = plt.colorbar(im, ax=ax,
                        orientation='vertical',
                        pad=0.02,
                        shrink=0.5,
                        ticks=[0,1,2])
    cbar.set_ticklabels(['Clear','Liquid','Ice'])
    cbar.set_label('Cloud Phase (CLP)', fontsize=12)

    plt.tight_layout()
    plt.savefig(save_path, bbox_inches='tight')
    plt.close(fig)

# 时次列表（UTC or 本地看需求）
hours = ['00:00', '03:00', '06:00', '09:00',
         '12:00', '15:00', '18:00', '21:00']
# 输出文件夹
out_dir = "clp_frames"
os.makedirs(out_dir, exist_ok=True)

# 1) 生成每一帧
frame_paths = []
for idx, hour in enumerate(hours):
    data = clp_all[idx, :, :]
    fname = os.path.join(out_dir, f"clp_{hour.replace(':','')}.png")
    plot_classification_frame(data, Lon2d, Lat2d,
                              timestamp=f"2022-01-01 {hour} UTC",
                              save_path=fname)
    frame_paths.append(fname)

# 2) 合成 GIF
with imageio.get_writer('clp_animation.gif', mode='I', duration=0.5) as writer:
    for fp in frame_paths:
        image = imageio.imread(fp)
        writer.append_data(image)

print("生成完成：clp_animation.gif")


/home/nvme/zhaolx/.conda/envs/lx/lib/python3.9/site-packages/cartopy/mpl/geoaxes.py:307: UserWarning: The colormap's 'bad' has been set, but in order to wrap pcolormesh across the map it must be fully transparent.
  return func(self, *args, **kwargs)
/home/nvme/zhaolx/.conda/envs/lx/lib/python3.9/site-packages/cartopy/mpl/geoaxes.py:307: UserWarning: The colormap's 'bad' has been set, but in order to wrap pcolormesh across the map it must be fully transparent.
  return func(self, *args, **kwargs)
/home/nvme/zhaolx/.conda/envs/lx/lib/python3.9/site-packages/cartopy/mpl/geoaxes.py:307: UserWarning: The colormap's 'bad' has been set, but in order to wrap pcolormesh across the map it must be fully transparent.
  return func(self, *args, **kwargs)
/home/nvme/zhaolx/.conda/envs/lx/lib/python3.9/site-packages/cartopy/mpl/geoaxes.py:307: UserWarning: The colormap's 'bad' has been set, but in order to wrap pcolormesh across the map it must be fully transparent.
  return func(self, *args, **kwar

生成完成：clp_animation.gif
